## Define the Objective of the Analysis

...

Analisi del sentiment relativo al dibattito sullo _Smart Working_ in Italia e individuazione di eventuali 'influencer' sul tema.

## Collect Data

In [1]:
# imports
import numpy as np
import pandas as pd
import tweepy
tweepy.__version__

import re
import string
import itertools
from collections import Counter
from datetime import datetime
from numpy.core.multiarray import result_type
import time

In [2]:
# insert the keys here
consumer_key = 'VPzjkqKl2y1uSTJQvnVqS9e1X' 
consumer_secret = 'STG2IzVMf65vPGeOvBQyzdeoKBExAr5sIkhOaBeDe2fnIN14vY'
access_token = '1508409949835214853-HIyZJ3oT32TijKsdNDhGFZEEQTWwau'
access_token_secret = 'uLcs9hUYmLdocxkaSfXo69Gii46TISu5qZj5F6f6fBfnW'

#### Tweets Download

The next step is creating an OAuthHandler instance. We pass our consumer key and access token which we defined above.

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

Next, we pass the OAuthHandler instance into the API method.

In [4]:
api = tweepy.API(auth, wait_on_rate_limit=True)

Tweets that contain a specific hashtag

In [ ]:
from tqdm.notebook import tqdm

import time
hashtag = '("#smartworking" OR "#remotework" OR "#lavoroagile")'

list_tweets = []

for i in range(10):
    for tweet in tweepy.Cursor(api.search_tweets, q=hashtag, count=50, lang='it').items(850):
      print('entering')
      full_text = api.get_status(tweet.id, tweet_mode='extended')._json['full_text']
      print(tweet.id)
      list_tweets.append([tweet.created_at, tweet.id, full_text, tweet.favorite_count, tweet.retweet_count, tweet.user.screen_name,
      tweet.user.location, tweet.retweeted, tweet.entities['user_mentions'], tweet.entities['hashtags']])
    for j in range(60):
        time.sleep(1)

# items is the maximum number of tweets to download.
# count is the number of tweets to return per page, up to a maximum of 100.

In [5]:
from tqdm.notebook import tqdm

import time
hashtag = '("#smartworking" OR "#remotework" OR "#lavoroagile")'

list_tweets = []

for tweet in tweepy.Cursor(api.search_tweets, q=hashtag, count=100, lang='it').items(10000):
  print('entering')
  full_text = api.get_status(tweet.id, tweet_mode='extended')._json['full_text']
  print(tweet.id)
  list_tweets.append([tweet.created_at, tweet.id, full_text, tweet.favorite_count, tweet.retweet_count, tweet.user.screen_name,
                      tweet.user.location, tweet.retweeted, tweet.entities['user_mentions'], tweet.entities['hashtags']])

# items is the maximum number of tweets to download.
# count is the number of tweets to return per page, up to a maximum of 100.

entering
1609873844600475648
entering
1609873750807252994
entering
1609873119241539584
entering
1609873116381040641
entering
1609873113231261697
entering
1609873101465976836
entering
1609866844503949312
entering
1609861037955682304
entering
1609860651932860416
entering
1609853259706941440
entering
1609852389179248641
entering
1609852313107144705
entering
1609848149358559232
entering
1609844412334411776
entering
1609843639366082560
entering
1609842700391714818
entering
1609840515846905858
entering
1609837891294826496
entering
1609836853460627459
entering
1609835815407951872
entering
1609835680573624321
entering
1609834419287740417
entering
1609834213251100674
entering
1609831986121539584
entering
1609831924679286784
entering
1609826066603950080
entering
1609818101046018048
entering
1609814958396608514
entering
1609811809258639360
entering
1609801031424016384
entering
1609799773682294784
entering
1609699318277017600
entering
1609664642955022336
entering
1609643604217053186
entering
16096

In [6]:
tweets = pd.DataFrame(list_tweets, columns=['date','id','text','like','n_rt','author','location','retweeted','user_mentions','hastags'])
tweets.head()

,date,id,text,like,n_rt,author,location,retweeted,user_mentions,hastags
0,2023-01-02 11:26:49+00:00,1609873844600475648,RT @mrmcphisto: @FrancescoLollo1 chi? ai patri...,0,1,lucague46760617,,False,"[{'screen_name': 'mrmcphisto', 'name': 'mrmcph...","[{'text': 'smartworking', 'indices': [74, 87]}]"
1,2023-01-02 11:26:26+00:00,1609873750807252994,RT @mrmcphisto: Stamattina un collega mi dice ...,0,1,lucague46760617,,False,"[{'screen_name': 'mrmcphisto', 'name': 'mrmcph...",[]
2,2023-01-02 11:23:56+00:00,1609873119241539584,"In sintesi, il #telelavoro è una modalità di l...",0,0,salvatopo,"Puglia, Italia",False,[],"[{'text': 'telelavoro', 'indices': [15, 26]}, ..."
3,2023-01-02 11:23:55+00:00,1609873116381040641,"Lo #smartworking può includere il telelavoro, ...",0,0,salvatopo,"Puglia, Italia",False,[],"[{'text': 'smartworking', 'indices': [3, 16]}]"
4,2023-01-02 11:23:54+00:00,1609873113231261697,"Lo #smartworking, al contrario, è una modalità...",0,0,salvatopo,"Puglia, Italia",False,[],"[{'text': 'smartworking', 'indices': [3, 16]}]"


In [7]:
len(list_tweets)

796

In [ ]:
# Turn list_tweet into a DataFrame changing column names
tweets = pd.DataFrame(list_tweets, columns=['date','id','text','like','n_rt','author','location','retweeted','user_mentions','hastags'])
tweets.to_csv('../data/SW.csv')

In [ ]:
tweets_loaded = pd.read_csv('../data/SW.csv')
tweets_loaded = tweets_loaded.drop('Unnamed: 0', axis=1)
tweets_loaded.head()

In [ ]:
tweets_loaded.shape

In [ ]:
tweets_loaded.columns

#### Data Pre-Processing

In [ ]:
# Remove duplicates
tweets_loaded.drop_duplicates(subset ="id", inplace = True)
tweets_loaded.reset_index(drop = True, inplace = True)
tweets_loaded.shape

In [ ]:
# Change date format
day = tweets['date'].dt.day
month = tweets['date'].dt.month
year = tweets['date'].dt.year

date = year.astype(str) + month.astype(str).str.zfill(2) + day.astype(str).str.zfill(2)
date = pd.to_datetime(date, format='%Y%m%d')
tweets.drop(columns = ['date'], inplace = True)
tweets['date'] = date

# Reorder columns
cols = tweets.columns.tolist()
cols = cols[-1:] + cols[:-1]
tweets = tweets[cols].copy()

print('Tweet per day:')
print()
print(tweets.groupby('date').count()['id'])
# print()
# print()
# print('Minimum Tweet ID per day:')
# print()
# print(tweets.groupby('date').min('id')['id'])
# print()
# print()
# print('Maximum Tweet ID per day:')
# print()
# print(tweets.groupby('date').max('id')['id'])

In [ ]:
# Create a DataFrame with the authors of the tweets and their respective frequency
freq_authors = tweets['author'].value_counts()
freq_authors.head()

In [ ]:
# Extract all hashtags from the full text
tweets['hashtags_list'] = tweets['text'].apply(lambda x: re.findall(r"#(\w+)", x))

# Extract all mentions from the full text
tweets['mentions'] = tweets['text'].apply(lambda x: re.findall(r"@(\w+)", x))

# \w matches any single letter, number or underscore (same as [a zA Z0 9_])

#### Data Cleaning

NLTK - Natural Language ToolKit is a platform for building Python programs to work with human language data. It provides easy to use interfaces to over 50 corpora and lexical resources, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning.

In [ ]:
import nltk
from nltk import FreqDist
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

Text Pre-Processing

In [ ]:
def preprocess_data(data):
    # remove numbers and turning words into lower case
    data = data.astype(str).str.replace('\d+','')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer = TweetTokenizer()
    
    # token lemmatization (ex. goes --> go)
    def lemmatize_text(text):
        return[(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    
    # remove punctuation
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)

In [ ]:
# apply preprocess_data function
pre_tweets = preprocess_data(tweets['text'])
tweets['text_proc'] = pre_tweets

# delete italian stopwords
stop_words = set(stopwords.words('italian'))
tweets['text_proc'] = tweets['text_proc'].apply(lambda x: [item for item in x if item not in stop_words])

## Analysis

### Social Content Analysis

#### _Sentiment Analysis_

When dealing with social media text, we usually want to identify urls, hashtags, smileys as separate objects and do not tokenize it to individual characters.

VADER

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

In [ ]:
sent_analyzer = SentimentIntensityAnalyzer()

In [ ]:
tweets[['text', 'text_proc']]

In [ ]:
tweets['scores'] = tweets['text'].apply(lambda Tweet:sent_analyzer.polarity_scores(Tweet))
tweets['compound'] = tweets['scores'].apply(lambda score_dict:score_dict['compound'])

tweets.head()

FEEL-IT: Emotion and Sentiment Classification for the Italian Language.

https://towardsdatascience.com/sentiment-analysis-and-emotion-recognition-in-italian-using-bert-92f5c8fe8a2

In [ ]:
from feel_it import EmotionClassifier, SentimentClassifier

sentiment_classifier = SentimentClassifier()
emotion_classifier = EmotionClassifier()

The feel-it-italian-sentiment model performs sentiment analysis on Italian. We fine-tuned the UmBERTo model on our new dataset (i.e., FEEL-IT) obtaining state-of-the-art performances on different benchmark corpora.

In [ ]:
print(sentiment_classifier.predict(tweets["text"].values.tolist()))

In [ ]:
tweets_sentiment = tweets.copy()

In [ ]:
text = tweets_sentiment['text']
li_sent = []
for i in range(0, text.shape[0]):
  sent = sentiment_classifier.predict([text[i]])
  li_sent.append(sent)
  if i % 5000 == 0:
    print('Riga',i,'su',text.shape[0])

tweets_sentiment['sentiment_BERT'] = [item for sublist in li_sent for item in sublist]
     
positive = []
negative = []

In [ ]:
positive = []
negative = []
ratio = []

for line in tweets_sentiment.values:
  sent = line[15]

  if sent == 'negative':
    positive.append(0)
    negative.append(1)
    ratio.append(-1)
  else:
    positive.append(1)
    negative.append(0)
    ratio.append(1)

In [ ]:
tweets_sentiment['positive'] = positive
tweets_sentiment['negative'] = negative
tweets_sentiment['ratio'] = ratio

In [ ]:
tweets_sentiment.head()

In [ ]:
tweets_sentiment.to_csv('../data/SW_sentiment.csv')

#### _Emotion Analysis_

Recognizing emotions in text is fundamental to get a better sense of how people are talking about something. People can talk about a new event, but positive/negative labels might not be enough.

The feel-it-italian-emotion model performs emotion classification (joy, fear, anger, sadness) on Italian. We fine-tuned the UmBERTo model on our new dataset (i.e., FEEL-IT) obtaining state-of-the-art performances on different benchmark corpora.

In [ ]:
print(emotion_classifier.predict(tweets["text"].values.tolist()))

In [ ]:
tweets_emotion = tweets.copy()

In [ ]:
text = tweets_emotion['text']
li_emotion = []
for i in range(0, text.shape[0]):
  emotion = emotion_classifier.predict([text[i]])
  li_emotion.append(emotion)
  if i % 5000 == 0:
    print('Riga',i,'su',text.shape[0])

tweets_emotion['emotion_BERT'] = [item for sublist in li_emotion for item in sublist]
     
anger = []
joy = []
fear = []
sadness = []

In [ ]:
anger = []
joy = []
fear = []
sadness = []

for line in tweets_emotion.values:
  emotion = line[15]

  if emotion == 'anger':
    anger.append(1)
    joy.append(0)
    fear.append(0)
    sadness.append(0)
  elif emotion == 'joy':
    anger.append(0)
    joy.append(1)
    fear.append(0)
    sadness.append(0)
  elif emotion == 'fear':
    anger.append(0)
    joy.append(0)
    fear.append(1)
    sadness.append(0)
  else:
    anger.append(0)
    joy.append(0)
    fear.append(0)
    sadness.append(1)

In [ ]:
tweets_emotion['anger'] = anger
tweets_emotion['joy'] = joy
tweets_emotion['fear'] = fear
tweets_emotion['sadness'] = sadness

In [ ]:
tweets_emotion.head()

In [ ]:
tweets_emotion.to_csv('../data/SW_emotion.csv')

---

#### Social Network Analysis

Measures of Centrality

Community Detection

## Visualization